# NYC Police Accountability RAG System

<a href="https://colab.research.google.com/github/droy9/nyc-officer-complaints-RAG/blob/main/complaintsrag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A modular Retrieval-Augmented Generation system for querying NYC police misconduct data.

## Architecture
```
┌─────────────────┐     ┌─────────────────┐     ┌─────────────────┐
│  Data Ingestors │ ──▶ │  Text Processor │ ──▶ │  Index Builder  │
│  (Extensible)   │     │  (Narratives)   │     │  (FAISS)        │
└─────────────────┘     └─────────────────┘     └─────────────────┘
                                                        │
┌─────────────────┐     ┌─────────────────┐             ▼
│   LLM Gateway   │ ◀── │   RAG Pipeline  │ ◀── ┌─────────────────┐
│   (Portkey)     │     │   (Orchestrator)│     │  Retriever      │
└─────────────────┘     └─────────────────┘     └─────────────────┘
```

In [ ]:
## 1. Installation & Setup

In [ ]:
# Install required packages
!pip install -q pandas faiss-cpu sentence-transformers portkey-ai

"""
Configuration & Environment Setup
==================================
Centralized configuration for the RAG system. All paths, model names,
and tunable parameters are defined here.
"""

import os
import logging
from dataclasses import dataclass, field
from typing import Optional
from pathlib import Path

# ============================================================================
# LOGGING SETUP
# ============================================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)-8s | %(name)s | %(message)s',
    datefmt='%H:%M:%S'
)
logger = logging.getLogger("rag")

# ============================================================================
# CONFIGURATION
# ============================================================================
@dataclass
class Config:
    """Central configuration for the RAG system."""
    
    # Data paths
    data_dir: str = "."
    raw_data_filename: str = "complaintclean.csv"
    processed_data_filename: str = "complaintclean_narrative.csv"
    index_filename: str = "complaints.faiss"
    metadata_filename: str = "complaints_meta.json"
    
    # Embedding model
    embedding_model: str = "sentence-transformers/all-MiniLM-L6-v2"
    embedding_dim: int = 384
    
    # Chunking parameters
    chunk_size: int = 512  # Characters per chunk (was 200, too small)
    chunk_overlap: int = 128  # Overlap between chunks
    
    # Retrieval parameters
    retrieval_k: int = 4  # Number of chunks to retrieve
    max_context_chars: int = 3000  # Max chars per retrieved chunk
    
    # LLM configuration
    llm_model: str = "@first-integrati-db9427/gemini-2.5-flash-lite"
    llm_max_tokens: int = 512
    
    # API keys (loaded from environment/Colab secrets)
    portkey_api_key: Optional[str] = field(default=None, repr=False)
    
    @property
    def raw_data_path(self) -> Path:
        return Path(self.data_dir) / self.raw_data_filename
    
    @property
    def processed_data_path(self) -> Path:
        return Path(self.data_dir) / self.processed_data_filename
    
    @property
    def index_path(self) -> Path:
        return Path(self.data_dir) / self.index_filename
    
    @property
    def metadata_path(self) -> Path:
        return Path(self.data_dir) / self.metadata_filename
    
    def load_api_keys(self):
        """Load API keys from Colab secrets or environment variables."""
        try:
            from google.colab import userdata
            self.portkey_api_key = userdata.get('PORT_KEY')
            logger.info("Loaded API key from Colab secrets")
        except (ImportError, Exception):
            self.portkey_api_key = os.getenv('PORTKEY_API_KEY')
            if self.portkey_api_key:
                logger.info("Loaded API key from environment variable")
        
        if not self.portkey_api_key:
            logger.warning("⚠️ No API key found. Set PORT_KEY in Colab secrets or PORTKEY_API_KEY env var.")
    
    def validate(self):
        """Validate configuration and check required files exist."""
        errors = []
        
        if not self.raw_data_path.exists():
            errors.append(f"Raw data file not found: {self.raw_data_path}")
        
        if errors:
            for err in errors:
                logger.error(err)
            return False
        
        logger.info("✓ Configuration validated successfully")
        return True

# Initialize global config
config = Config()
config.load_api_keys()

print(f"""
╔══════════════════════════════════════════════════════════════╗
║          NYC Police Accountability RAG System                ║
╠══════════════════════════════════════════════════════════════╣
║  Embedding Model: {config.embedding_model:<40} ║
║  Chunk Size:      {config.chunk_size:<40} ║
║  Retrieval K:     {config.retrieval_k:<40} ║
║  LLM Model:       {config.llm_model[:40]:<40} ║
║  API Key:         {'✓ Loaded' if config.portkey_api_key else '✗ Missing':<40} ║
╚══════════════════════════════════════════════════════════════╝
""")

## 2. Data Ingestion Layer (Extensible)

This layer uses an abstract base class pattern to support multiple data sources.
To add a new data source:
1. Create a new class inheriting from `DataIngestor`
2. Implement `load()`, `validate()`, and `to_narrative()` methods
3. Register it in the `INGESTORS` dictionary

In [ ]:
"""
Data Ingestion Layer
====================
Extensible framework for ingesting multiple data sources.
Each data source has its own ingestor class that handles loading,
validation, and narrative generation.
"""

from abc import ABC, abstractmethod
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
import pandas as pd

# ============================================================================
# BASE INGESTOR CLASS
# ============================================================================
@dataclass
class IngestedRecord:
    """Standardized record format across all data sources."""
    source_type: str  # e.g., "ccrb_complaint", "sqf_encounter", "gang_database"
    record_id: str
    narrative: str
    metadata: Dict[str, Any]  # Structured fields for filtering


class DataIngestor(ABC):
    """
    Abstract base class for data ingestors.
    
    To add a new data source:
    1. Subclass DataIngestor
    2. Implement load(), validate(), and to_narrative()
    3. Register in INGESTORS dict at bottom of this cell
    """
    
    def __init__(self, config: Config):
        self.config = config
        self.df: Optional[pd.DataFrame] = None
        self.logger = logging.getLogger(f"rag.ingestor.{self.source_name}")
    
    @property
    @abstractmethod
    def source_name(self) -> str:
        """Unique identifier for this data source."""
        pass
    
    @property
    @abstractmethod
    def required_columns(self) -> List[str]:
        """List of columns that must exist in the source data."""
        pass
    
    @abstractmethod
    def load(self, path: str) -> pd.DataFrame:
        """Load data from the specified path."""
        pass
    
    @abstractmethod
    def to_narrative(self, row: pd.Series) -> str:
        """Convert a single row to narrative text."""
        pass
    
    def validate(self, df: pd.DataFrame) -> bool:
        """Validate that DataFrame has required structure."""
        missing = set(self.required_columns) - set(df.columns)
        if missing:
            self.logger.error(f"Missing required columns: {missing}")
            return False
        return True
    
    def safe_get(self, row: pd.Series, col: str, default: str = "Unknown") -> str:
        """Safely get a value from a row, handling missing/null values."""
        val = row.get(col)
        if pd.isna(val) or val == "Unlisted" or val == "" or val is None:
            return default
        return str(val)
    
    def ingest(self, path: str, limit: Optional[int] = None) -> List[IngestedRecord]:
        """
        Full ingestion pipeline: load → validate → transform.
        
        Args:
            path: Path to the data file
            limit: Optional limit on number of records (for testing)
        
        Returns:
            List of IngestedRecord objects
        """
        self.logger.info(f"Loading data from {path}")
        
        df = self.load(path)
        
        if limit:
            self.logger.info(f"Limiting to {limit} records (testing mode)")
            df = df.head(limit)
        
        if not self.validate(df):
            raise ValueError(f"Validation failed for {self.source_name}")
        
        self.df = df
        records = []
        
        for idx, row in df.iterrows():
            try:
                narrative = self.to_narrative(row)
                record = IngestedRecord(
                    source_type=self.source_name,
                    record_id=str(row.get(self.id_column, idx)),
                    narrative=narrative,
                    metadata=self.extract_metadata(row)
                )
                records.append(record)
            except Exception as e:
                self.logger.warning(f"Error processing row {idx}: {e}")
                continue
        
        self.logger.info(f"✓ Ingested {len(records)} records from {self.source_name}")
        return records
    
    @property
    def id_column(self) -> str:
        """Column to use as record ID. Override if different."""
        return self.required_columns[0] if self.required_columns else "index"
    
    def extract_metadata(self, row: pd.Series) -> Dict[str, Any]:
        """Extract structured metadata for filtering. Override for custom fields."""
        return {}


# ============================================================================
# CCRB COMPLAINTS INGESTOR
# ============================================================================
class CCRBComplaintIngestor(DataIngestor):
    """Ingestor for CCRB complaint data."""
    
    @property
    def source_name(self) -> str:
        return "ccrb_complaint"
    
    @property
    def required_columns(self) -> List[str]:
        return [
            "complaint_id", "incident_date", "first_name", "last_name",
            "location_type", "contact_reason", "fado_type", "allegation_cat",
            "contact_outcome", "ccrb_disposition", "penalty_rec", "status_cat",
            "officer_race", "officer_gender", "days_on_force",
            "officer_incident_rank", "officer_current_rank",
            "impacted_race", "impacted_gender"
        ]
    
    @property
    def id_column(self) -> str:
        return "complaint_id"
    
    def load(self, path: str) -> pd.DataFrame:
        return pd.read_csv(path, low_memory=False)
    
    def to_narrative(self, row: pd.Series) -> str:
        """Convert a CCRB complaint row to narrative text."""
        
        # Incident overview
        date = self.safe_get(row, 'incident_date', 'an unknown date')
        date_str = f"On {date}" if date != 'an unknown date' else "On an unknown date"
        
        first_name = self.safe_get(row, 'first_name')
        last_name = self.safe_get(row, 'last_name')
        complaint_id = self.safe_get(row, 'complaint_id', 'Unknown ID')
        
        if first_name != "Unknown" and last_name != "Unknown":
            officer_str = f"complaint {complaint_id} was filed against Officer {first_name} {last_name}"
        else:
            officer_str = f"complaint {complaint_id} was filed against an unidentified officer"
        
        location = self.safe_get(row, 'location_type', 'an unknown location')
        location_str = f"at a(n) {location}" if location != 'an unknown location' else "at an unknown location"
        
        reason = self.safe_get(row, 'contact_reason', 'unspecified reason')
        fado = self.safe_get(row, 'fado_type', 'Unknown')
        allegation = self.safe_get(row, 'allegation_cat', 'Unknown')
        outcome = self.safe_get(row, 'contact_outcome', 'Unknown')
        disposition = self.safe_get(row, 'ccrb_disposition', 'Unknown')
        penalty = self.safe_get(row, 'penalty_rec', 'Not Applicable')
        status = self.safe_get(row, 'status_cat', 'Unknown')
        
        overview = (
            f"{date_str}, {officer_str} {location_str} for the reason: \"{reason}\". "
            f"The allegation was classified as FADO type '{fado}' with specific allegation '{allegation}'. "
            f"Contact outcome: {outcome}. CCRB disposition: \"{disposition}\". "
            f"{'Penalty: ' + penalty + '.' if penalty != 'Not Applicable' else 'No penalty received.'} "
            f"Status: {status}."
        )
        
        # Officer demographics
        o_race = self.safe_get(row, 'officer_race')
        o_gender = self.safe_get(row, 'officer_gender')
        o_days = self.safe_get(row, 'days_on_force')
        o_inc_rank = self.safe_get(row, 'officer_incident_rank')
        o_cur_rank = self.safe_get(row, 'officer_current_rank')
        
        officer_info = (
            f"Officer demographics: Race: {o_race}, Gender: {o_gender}. "
            f"Days on force: {o_days}. "
            f"Rank at incident: {o_inc_rank}, Current rank: {o_cur_rank}."
        )
        
        # Impacted party demographics
        i_race = self.safe_get(row, 'impacted_race')
        i_gender = self.safe_get(row, 'impacted_gender')
        
        impacted_info = f"Impacted party: Race: {i_race}, Gender: {i_gender}."
        
        return f"{overview}\n\n{officer_info}\n\n{impacted_info}"
    
    def extract_metadata(self, row: pd.Series) -> Dict[str, Any]:
        """Extract structured metadata for filtering."""
        return {
            "complaint_id": self.safe_get(row, 'complaint_id'),
            "incident_date": self.safe_get(row, 'incident_date'),
            "officer_name": f"{self.safe_get(row, 'first_name')} {self.safe_get(row, 'last_name')}",
            "fado_type": self.safe_get(row, 'fado_type'),
            "allegation_cat": self.safe_get(row, 'allegation_cat'),
            "ccrb_disposition": self.safe_get(row, 'ccrb_disposition'),
            "officer_race": self.safe_get(row, 'officer_race'),
            "impacted_race": self.safe_get(row, 'impacted_race'),
        }


# ============================================================================
# STOP, QUESTION & FRISK INGESTOR (TEMPLATE FOR FUTURE)
# ============================================================================
class SQFIngestor(DataIngestor):
    """
    Template ingestor for Stop, Question & Frisk data.
    TODO: Implement when SQF data integration begins.
    """
    
    @property
    def source_name(self) -> str:
        return "sqf_encounter"
    
    @property
    def required_columns(self) -> List[str]:
        # Placeholder - update with actual SQF columns
        return ["stop_id", "date", "precinct", "suspect_race", "suspect_age"]
    
    def load(self, path: str) -> pd.DataFrame:
        raise NotImplementedError("SQF ingestor not yet implemented")
    
    def to_narrative(self, row: pd.Series) -> str:
        raise NotImplementedError("SQF ingestor not yet implemented")


# ============================================================================
# INGESTOR REGISTRY
# ============================================================================
INGESTORS: Dict[str, type] = {
    "ccrb": CCRBComplaintIngestor,
    "sqf": SQFIngestor,
    # Add new ingestors here as they are implemented
}

def get_ingestor(source_type: str, config: Config) -> DataIngestor:
    """Factory function to get an ingestor by type."""
    if source_type not in INGESTORS:
        raise ValueError(f"Unknown ingestor type: {source_type}. Available: {list(INGESTORS.keys())}")
    return INGESTORS[source_type](config)

# Quick test
logger.info(f"Available ingestors: {list(INGESTORS.keys())}")


In [ ]:
## 3. Text Processing & Chunking

Handles text chunking with configurable size and overlap.
The `TextProcessor` class provides utilities for preparing text for embedding.

In [ ]:
"""
Text Processing & Chunking
===========================
Utilities for preparing text for embedding.
"""

from typing import List, Tuple
import hashlib


class TextProcessor:
    """
    Handles text chunking and preprocessing for the RAG pipeline.
    """
    
    def __init__(self, chunk_size: int = 512, chunk_overlap: int = 128):
        """
        Initialize the text processor.
        
        Args:
            chunk_size: Maximum characters per chunk
            chunk_overlap: Characters of overlap between consecutive chunks
        """
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.logger = logging.getLogger("rag.text_processor")
    
    def chunk_text(self, text: str) -> List[str]:
        """
        Split text into overlapping chunks.
        
        Args:
            text: The text to chunk
            
        Returns:
            List of text chunks
        """
        text = str(text).strip()
        
        if len(text) <= self.chunk_size:
            return [text] if text else []
        
        chunks = []
        start = 0
        
        while start < len(text):
            end = start + self.chunk_size
            chunk = text[start:end]
            
            # Try to break at sentence boundary if possible
            if end < len(text):
                # Look for sentence-ending punctuation near the end
                for punct in ['. ', '! ', '? ', '\n']:
                    last_punct = chunk.rfind(punct)
                    if last_punct > self.chunk_size * 0.5:  # Only if in latter half
                        chunk = chunk[:last_punct + 1]
                        end = start + len(chunk)
                        break
            
            chunks.append(chunk.strip())
            
            if end >= len(text):
                break
            
            start = end - self.chunk_overlap
        
        return chunks
    
    def prepare_documents(
        self, 
        records: List[IngestedRecord]
    ) -> Tuple[List[str], List[dict]]:
        """
        Prepare documents and metadata for indexing.
        
        Args:
            records: List of IngestedRecord objects
            
        Returns:
            Tuple of (documents, metadata) where each document is a chunk
        """
        documents = []
        metadata = []
        
        for record in records:
            chunks = self.chunk_text(record.narrative)
            
            for chunk_idx, chunk in enumerate(chunks):
                # Prepend record ID to each chunk for context
                doc = f"[{record.source_type}:{record.record_id}]\n\n{chunk}"
                documents.append(doc)
                
                meta = {
                    "source_type": record.source_type,
                    "record_id": record.record_id,
                    "chunk_idx": chunk_idx,
                    "total_chunks": len(chunks),
                    "content": doc,  # Store content for retrieval
                    **record.metadata
                }
                metadata.append(meta)
        
        self.logger.info(f"Prepared {len(documents)} chunks from {len(records)} records")
        return documents, metadata
    
    @staticmethod
    def compute_hash(text: str) -> str:
        """Compute MD5 hash of text for cache keys."""
        return hashlib.md5(text.encode()).hexdigest()[:12]


# Initialize with config
text_processor = TextProcessor(
    chunk_size=config.chunk_size,
    chunk_overlap=config.chunk_overlap
)

# Quick test
test_text = "This is a test. " * 50
test_chunks = text_processor.chunk_text(test_text)
logger.info(f"Test chunking: {len(test_text)} chars → {len(test_chunks)} chunks")

In [ ]:
## 4. Embedding & Index Building

The `IndexBuilder` handles:
- Embedding generation using sentence-transformers
- FAISS index creation and persistence
- Metadata storage for retrieval

In [ ]:
"""
Embedding & Index Building
===========================
Handles embedding generation and FAISS index management.
"""

import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from typing import List, Optional
import time


class IndexBuilder:
    """
    Builds and manages the FAISS vector index.
    
    Supports:
    - Batch embedding for memory efficiency
    - Index persistence (save/load)
    - Multiple index types (Flat, IVF for scalability)
    """
    
    def __init__(self, config: Config):
        self.config = config
        self.logger = logging.getLogger("rag.index_builder")
        self.model: Optional[SentenceTransformer] = None
        self.index: Optional[faiss.Index] = None
        self.metadata: List[dict] = []
    
    def load_embedding_model(self) -> SentenceTransformer:
        """Load the embedding model (lazy loading)."""
        if self.model is None:
            self.logger.info(f"Loading embedding model: {self.config.embedding_model}")
            self.model = SentenceTransformer(self.config.embedding_model)
            self.logger.info("✓ Embedding model loaded")
        return self.model
    
    def embed_documents(
        self, 
        documents: List[str], 
        batch_size: int = 256,
        show_progress: bool = True
    ) -> np.ndarray:
        """
        Embed documents in batches for memory efficiency.
        
        Args:
            documents: List of text documents to embed
            batch_size: Number of documents to embed at once
            show_progress: Whether to log progress
            
        Returns:
            Numpy array of embeddings (normalized for cosine similarity)
        """
        model = self.load_embedding_model()
        
        all_embeddings = []
        total_batches = (len(documents) + batch_size - 1) // batch_size
        
        start_time = time.time()
        
        for i in range(0, len(documents), batch_size):
            batch = documents[i:i + batch_size]
            batch_num = i // batch_size + 1
            
            if show_progress and batch_num % 10 == 0:
                elapsed = time.time() - start_time
                rate = i / elapsed if elapsed > 0 else 0
                self.logger.info(
                    f"Embedding batch {batch_num}/{total_batches} "
                    f"({rate:.0f} docs/sec)"
                )
            
            embeddings = model.encode(
                batch,
                convert_to_numpy=True,
                normalize_embeddings=True,  # Required for cosine similarity
                show_progress_bar=False
            )
            all_embeddings.append(embeddings)
        
        result = np.vstack(all_embeddings)
        
        elapsed = time.time() - start_time
        self.logger.info(
            f"✓ Embedded {len(documents)} documents in {elapsed:.1f}s "
            f"({len(documents)/elapsed:.0f} docs/sec)"
        )
        
        return result
    
    def build_index(
        self, 
        documents: List[str], 
        metadata: List[dict],
        use_ivf: bool = False,
        nlist: int = 100
    ) -> faiss.Index:
        """
        Build a FAISS index from documents.
        
        Args:
            documents: List of text documents
            metadata: List of metadata dicts for each document
            use_ivf: Whether to use IVF index (faster for large datasets)
            nlist: Number of clusters for IVF index
            
        Returns:
            FAISS index
        """
        if len(documents) != len(metadata):
            raise ValueError(
                f"Document count ({len(documents)}) != metadata count ({len(metadata)})"
            )
        
        # Generate embeddings
        embeddings = self.embed_documents(documents)
        dim = embeddings.shape[1]
        
        # Create index
        if use_ivf and len(documents) > 10000:
            # IVF index for large datasets (faster queries, needs training)
            self.logger.info(f"Building IVF index with {nlist} clusters")
            quantizer = faiss.IndexFlatIP(dim)
            self.index = faiss.IndexIVFFlat(quantizer, dim, nlist)
            self.index.train(embeddings)
            self.index.nprobe = min(10, nlist)  # Search 10 clusters
        else:
            # Flat index (exact search, good for <100k documents)
            self.logger.info("Building Flat index (exact search)")
            self.index = faiss.IndexFlatIP(dim)
        
        # Add embeddings
        self.index.add(embeddings.astype(np.float32))
        self.metadata = metadata
        
        self.logger.info(f"✓ Built index with {self.index.ntotal} vectors")
        
        return self.index
    
    def save(self, index_path: str = None, metadata_path: str = None):
        """Save index and metadata to disk."""
        index_path = index_path or str(self.config.index_path)
        metadata_path = metadata_path or str(self.config.metadata_path)
        
        if self.index is None:
            raise ValueError("No index to save. Call build_index() first.")
        
        faiss.write_index(self.index, index_path)
        with open(metadata_path, 'w') as f:
            json.dump(self.metadata, f)
        
        self.logger.info(f"✓ Saved index to {index_path}")
        self.logger.info(f"✓ Saved metadata to {metadata_path}")
    
    def load(self, index_path: str = None, metadata_path: str = None) -> bool:
        """Load index and metadata from disk."""
        index_path = index_path or str(self.config.index_path)
        metadata_path = metadata_path or str(self.config.metadata_path)
        
        try:
            self.index = faiss.read_index(index_path)
            with open(metadata_path, 'r') as f:
                self.metadata = json.load(f)
            
            self.logger.info(
                f"✓ Loaded index with {self.index.ntotal} vectors "
                f"and {len(self.metadata)} metadata entries"
            )
            return True
            
        except FileNotFoundError as e:
            self.logger.warning(f"Could not load index: {e}")
            return False


# Initialize builder
index_builder = IndexBuilder(config)
logger.info("IndexBuilder initialized")


In [ ]:
"""
Retrieval Engine
=================
Handles semantic search with caching, filtering, and metrics.
"""

from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional, Callable
from functools import lru_cache
import time


@dataclass
class RetrievalResult:
    """A single retrieval result with score and metadata."""
    score: float
    content: str
    metadata: Dict[str, Any]
    
    def __repr__(self):
        return f"RetrievalResult(score={self.score:.3f}, record_id={self.metadata.get('record_id', 'N/A')})"


@dataclass
class RetrievalMetrics:
    """Metrics for a retrieval operation."""
    query: str
    num_results: int
    top_score: float
    avg_score: float
    retrieval_time_ms: float
    timestamp: str = field(default_factory=lambda: time.strftime("%Y-%m-%d %H:%M:%S"))


class Retriever:
    """
    Semantic retrieval engine with caching and filtering support.
    
    Features:
    - Cached query embeddings
    - Metadata filtering
    - Retrieval metrics
    - Configurable scoring thresholds
    """
    
    def __init__(self, index_builder: IndexBuilder, config: Config):
        self.index_builder = index_builder
        self.config = config
        self.logger = logging.getLogger("rag.retriever")
        self._query_cache: Dict[str, np.ndarray] = {}
        self.metrics_history: List[RetrievalMetrics] = []
    
    def _embed_query(self, query: str) -> np.ndarray:
        """Embed a query with caching."""
        cache_key = TextProcessor.compute_hash(query)
        
        if cache_key not in self._query_cache:
            model = self.index_builder.load_embedding_model()
            embedding = model.encode(
                [query], 
                normalize_embeddings=True
            ).astype(np.float32)
            self._query_cache[cache_key] = embedding
            
            # Limit cache size
            if len(self._query_cache) > 1000:
                # Remove oldest entries (simple FIFO)
                oldest_key = next(iter(self._query_cache))
                del self._query_cache[oldest_key]
        
        return self._query_cache[cache_key]
    
    def retrieve(
        self,
        query: str,
        k: int = None,
        filters: Optional[Dict[str, Any]] = None,
        min_score: float = 0.0
    ) -> List[RetrievalResult]:
        """
        Retrieve relevant documents for a query.
        
        Args:
            query: The search query
            k: Number of results to return (default from config)
            filters: Optional metadata filters (e.g., {"fado_type": "Force"})
            min_score: Minimum similarity score threshold
            
        Returns:
            List of RetrievalResult objects sorted by score
        """
        k = k or self.config.retrieval_k
        start_time = time.time()
        
        if self.index_builder.index is None:
            raise ValueError("No index loaded. Call index_builder.load() or build_index() first.")
        
        # Embed query
        query_embedding = self._embed_query(query)
        
        # Search with over-retrieval if filtering
        search_k = k * 3 if filters else k
        distances, indices = self.index_builder.index.search(query_embedding, search_k)
        
        results = []
        for idx, score in zip(indices[0], distances[0]):
            if idx == -1:  # FAISS returns -1 for missing results
                continue
            
            if score < min_score:
                continue
            
            metadata = self.index_builder.metadata[idx]
            
            # Apply filters
            if filters:
                match = all(
                    metadata.get(key) == value 
                    for key, value in filters.items()
                )
                if not match:
                    continue
            
            # Get content (stored in metadata or reconstruct)
            content = metadata.get("content", f"[Record: {metadata.get('record_id', idx)}]")
            
            results.append(RetrievalResult(
                score=float(score),
                content=content[:self.config.max_context_chars],
                metadata=metadata
            ))
            
            if len(results) >= k:
                break
        
        # Calculate metrics
        elapsed_ms = (time.time() - start_time) * 1000
        metrics = RetrievalMetrics(
            query=query[:100],
            num_results=len(results),
            top_score=results[0].score if results else 0.0,
            avg_score=sum(r.score for r in results) / len(results) if results else 0.0,
            retrieval_time_ms=elapsed_ms
        )
        self.metrics_history.append(metrics)
        
        self.logger.info(
            f"Retrieved {len(results)} results in {elapsed_ms:.1f}ms "
            f"(top score: {metrics.top_score:.3f})"
        )
        
        return results
    
    def get_context_block(
        self, 
        results: List[RetrievalResult],
        include_scores: bool = False
    ) -> str:
        """Format retrieval results as a context block for the LLM."""
        blocks = []
        for i, result in enumerate(results, 1):
            header = f"[{i}] Record: {result.metadata.get('record_id', 'Unknown')}"
            if include_scores:
                header += f" (relevance: {result.score:.2f})"
            blocks.append(f"{header}\n{result.content}")
        
        return "\n\n---\n\n".join(blocks)
    
    def get_citations(self, results: List[RetrievalResult]) -> str:
        """Generate citation string for results."""
        citations = []
        for i, result in enumerate(results, 1):
            meta = result.metadata
            citations.append(
                f"[{i}] {meta.get('source_type', 'unknown')}:{meta.get('record_id', 'N/A')} "
                f"(chunk {meta.get('chunk_idx', 0)+1}/{meta.get('total_chunks', 1)}, "
                f"score: {result.score:.3f})"
            )
        return "\n".join(citations)
    
    def clear_cache(self):
        """Clear the query embedding cache."""
        self._query_cache.clear()
        self.logger.info("Query cache cleared")


# Initialize retriever (will be connected to index after loading)
retriever = Retriever(index_builder, config)
logger.info("Retriever initialized")


Bye!


## 5. LLM Integration

Handles communication with the LLM via Portkey gateway.
Includes error handling, retry logic, and response parsing.


In [ ]:
"""
LLM Gateway
============
Handles all communication with the LLM API via Portkey.
Includes error handling, retry logic, and rate limiting.
"""

from collections import deque


@dataclass
class LLMResponse:
    """Structured response from the LLM."""
    content: str
    model: str
    usage: Optional[Dict[str, int]] = None
    latency_ms: float = 0.0
    success: bool = True
    error: Optional[str] = None


class RateLimiter:
    """Simple rate limiter for API calls."""
    
    def __init__(self, max_calls: int = 10, period_seconds: int = 60):
        self.max_calls = max_calls
        self.period = period_seconds
        self.calls: deque = deque()
    
    def wait_if_needed(self):
        """Block if rate limit would be exceeded."""
        now = time.time()
        while self.calls and self.calls[0] < now - self.period:
            self.calls.popleft()
        if len(self.calls) >= self.max_calls:
            sleep_time = self.calls[0] + self.period - now
            if sleep_time > 0:
                time.sleep(sleep_time)
        self.calls.append(time.time())


class LLMGateway:
    """Gateway for LLM API calls via Portkey with retry and rate limiting."""
    
    def __init__(self, config: Config):
        self.config = config
        self.logger = logging.getLogger("rag.llm_gateway")
        self.client = None
        self.rate_limiter = RateLimiter(max_calls=10, period_seconds=60)
        self._initialize_client()
    
    def _initialize_client(self):
        if not self.config.portkey_api_key:
            self.logger.warning("No API key configured. LLM calls will fail.")
            return
        try:
            from portkey_ai import Portkey
            self.client = Portkey(api_key=self.config.portkey_api_key)
            self.logger.info("✓ Portkey client initialized")
        except ImportError:
            self.logger.error("portkey_ai not installed. Run: pip install portkey-ai")
        except Exception as e:
            self.logger.error(f"Failed to initialize Portkey client: {e}")
    
    def generate(self, system_prompt: str, user_prompt: str, max_tokens: int = None,
                 temperature: float = 0.7, max_retries: int = 3) -> LLMResponse:
        if self.client is None:
            return LLMResponse(content="", model=self.config.llm_model, success=False,
                             error="LLM client not initialized. Check API key.")
        
        max_tokens = max_tokens or self.config.llm_max_tokens
        
        for attempt in range(max_retries):
            try:
                self.rate_limiter.wait_if_needed()
                start_time = time.time()
                
                response = self.client.chat.completions.create(
                    model=self.config.llm_model,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt},
                    ],
                    max_tokens=max_tokens,
                    temperature=temperature,
                )
                
                latency_ms = (time.time() - start_time) * 1000
                
                if not response.choices:
                    raise ValueError("No choices in response")
                
                content = response.choices[0].message.content or ""
                usage = None
                if hasattr(response, 'usage') and response.usage:
                    usage = {
                        "prompt_tokens": getattr(response.usage, 'prompt_tokens', 0),
                        "completion_tokens": getattr(response.usage, 'completion_tokens', 0),
                        "total_tokens": getattr(response.usage, 'total_tokens', 0),
                    }
                
                self.logger.info(f"LLM response: {latency_ms:.0f}ms ({len(content)} chars)")
                return LLMResponse(content=content, model=self.config.llm_model,
                                 usage=usage, latency_ms=latency_ms, success=True)
                
            except Exception as e:
                self.logger.warning(f"LLM call failed (attempt {attempt + 1}/{max_retries}): {e}")
                if attempt < max_retries - 1:
                    time.sleep(2 ** attempt)
                else:
                    return LLMResponse(content="", model=self.config.llm_model, success=False,
                                     error=f"LLM call failed after {max_retries} attempts: {e}")
        
        return LLMResponse(content="", model=self.config.llm_model, success=False,
                         error="Unexpected error")


SYSTEM_PROMPTS = {
    "default": """You are a helpful assistant for the NYC Police Accountability Intelligence system.
You help users understand police misconduct data from the CCRB database.

When answering:
1. Use ONLY the provided CONTEXT to answer
2. Cite specific complaint IDs
3. If context is insufficient, state what's missing
4. Be precise, avoid speculation
5. Remember: Allegations are not convictions. Note disposition status.""",

    "summary": """Summarize police misconduct patterns from the context.
Focus on patterns, trends, and statistics. Cite complaint IDs.""",
}

llm_gateway = LLMGateway(config)
logger.info("LLM Gateway initialized")


# ============================================================================
# RAG PIPELINE ORCHESTRATOR
# ============================================================================
@dataclass
class RAGResponse:
    """Complete response from the RAG pipeline."""
    answer: str
    citations: str
    retrieval_results: List[RetrievalResult]
    llm_response: LLMResponse
    success: bool = True
    error: Optional[str] = None


class RAGPipeline:
    """Main orchestrator for the RAG system."""
    
    def __init__(self, config, index_builder, retriever, llm_gateway, text_processor):
        self.config = config
        self.index_builder = index_builder
        self.retriever = retriever
        self.llm_gateway = llm_gateway
        self.text_processor = text_processor
        self.logger = logging.getLogger("rag.pipeline")
        self._is_initialized = False
    
    def build_index_from_source(self, source_type: str = "ccrb", data_path: str = None,
                                 limit: Optional[int] = None, save: bool = True) -> int:
        """Build search index from data source."""
        data_path = data_path or str(self.config.raw_data_path)
        ingestor = get_ingestor(source_type, self.config)
        records = ingestor.ingest(data_path, limit=limit)
        documents, metadata = self.text_processor.prepare_documents(records)
        self.index_builder.build_index(documents, metadata)
        if save:
            self.index_builder.save()
        self._is_initialized = True
        self.logger.info(f"✓ Pipeline initialized with {len(documents)} documents")
        return len(documents)
    
    def load_index(self) -> bool:
        """Load existing index from disk."""
        success = self.index_builder.load()
        if success:
            self._is_initialized = True
            self.logger.info("✓ Pipeline loaded from existing index")
        return success
    
    def query(self, question: str, k: int = None, filters: Optional[Dict[str, Any]] = None,
              system_prompt_key: str = "default") -> RAGResponse:
        """Process a user query through the full RAG pipeline."""
        if not self._is_initialized:
            return RAGResponse(answer="", citations="", retrieval_results=[],
                             llm_response=LLMResponse(content="", model="", success=False),
                             success=False, error="Pipeline not initialized.")
        
        self.logger.info(f"Processing: {question[:50]}...")
        
        try:
            results = self.retriever.retrieve(question, k=k, filters=filters)
        except Exception as e:
            return RAGResponse(answer="", citations="", retrieval_results=[],
                             llm_response=LLMResponse(content="", model="", success=False),
                             success=False, error=f"Retrieval failed: {e}")
        
        if not results:
            return RAGResponse(answer="No relevant information found.", citations="",
                             retrieval_results=[], 
                             llm_response=LLMResponse(content="", model="", success=True),
                             success=True)
        
        context_block = self.retriever.get_context_block(results)
        citations = self.retriever.get_citations(results)
        system_prompt = SYSTEM_PROMPTS.get(system_prompt_key, SYSTEM_PROMPTS["default"])
        
        user_prompt = f"QUESTION: {question}\n\nCONTEXT:\n{context_block}\n\nProvide a comprehensive answer citing complaint IDs."
        llm_response = self.llm_gateway.generate(system_prompt, user_prompt)
        
        return RAGResponse(answer=llm_response.content, citations=citations,
                          retrieval_results=results, llm_response=llm_response,
                          success=llm_response.success, error=llm_response.error)


pipeline = RAGPipeline(config, index_builder, retriever, llm_gateway, text_processor)
logger.info("RAG Pipeline initialized")


## 7. Build Index (Run Once)

Run this cell to ingest data and build the search index.
Set `limit` to a small number for testing, or `None` for full dataset.


In [ ]:
# Build index from CCRB complaint data
# Set limit=100 for quick testing, limit=None for full dataset

try:
    # First try to load existing index
    if pipeline.load_index():
        print("✓ Loaded existing index")
    else:
        # Build new index
        print("Building new index from complaintclean.csv...")
        num_docs = pipeline.build_index_from_source(
            source_type="ccrb",
            limit=None,  # Set to 100 for testing, None for full dataset
            save=True
        )
        print(f"✓ Built index with {num_docs} document chunks")
except Exception as e:
    print(f"Error: {e}")
    print("\nMake sure 'complaintclean.csv' exists in the current directory.")


## 8. Interactive Chat

Query the RAG system with natural language questions about police misconduct data.


In [ ]:
def ask(question: str, k: int = 4, filters: Dict = None) -> None:
    """
    Convenience function to query the RAG system.
    
    Args:
        question: Your question about police misconduct data
        k: Number of documents to retrieve (default 4)
        filters: Optional filters e.g. {"fado_type": "Force"}
    
    Examples:
        ask("How many complaints involved excessive force?")
        ask("What are the most common allegation types?")
        ask("Show me complaints with substantiated outcomes")
    """
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print('='*60)
    
    response = pipeline.query(question, k=k, filters=filters)
    
    if not response.success:
        print(f"\n❌ Error: {response.error}")
        return
    
    print(f"\n📝 Answer:\n{response.answer}")
    print(f"\n📚 Sources:\n{response.citations}")
    if response.llm_response.latency_ms > 0:
        print(f"\n⏱️ LLM Latency: {response.llm_response.latency_ms:.0f}ms")


print("RAG System Ready!")
print("\nUse ask('your question') to query the system.")
print("\nExample queries:")
print('  ask("How many complaints involved excessive force?")')
print('  ask("What are the most common allegation types?")')
print('  ask("Show me complaints against officers with multiple allegations")')


In [ ]:
# Example query - uncomment to test
# ask("What types of complaints are most common in this dataset?")


In [ ]:
# Interactive chat loop - uncomment to start
def chat():
    """Interactive chat loop for the RAG system."""
    print("\n" + "="*60)
    print("NYC Police Accountability RAG Chat")
    print("Type 'quit' to exit, 'clear' to clear cache")
    print("="*60 + "\n")
    
    while True:
        try:
            question = input("\nYou: ").strip()
            
            if not question:
                continue
            
            if question.lower() in ['quit', 'exit', 'q']:
                print("\nGoodbye!")
                break
            
            if question.lower() == 'clear':
                retriever.clear_cache()
                print("Cache cleared.")
                continue
            
            ask(question)
            
        except KeyboardInterrupt:
            print("\n\nGoodbye!")
            break

# Uncomment to start interactive chat:
# chat()


---
## Adding New Data Sources

To integrate a new data source (e.g., Stop-Question-Frisk, Gang Database):

1. **Create an Ingestor Class** in Cell 5:
```python
class GangDatabaseIngestor(DataIngestor):
    @property
    def source_name(self) -> str:
        return "gang_database"
    
    @property
    def required_columns(self) -> List[str]:
        return ["entry_id", "date_added", "criteria", ...]
    
    def load(self, path: str) -> pd.DataFrame:
        return pd.read_csv(path)
    
    def to_narrative(self, row: pd.Series) -> str:
        # Convert row to narrative text
        return f"On {row['date_added']}, ..."
```

2. **Register in INGESTORS dict**:
```python
INGESTORS = {
    "ccrb": CCRBComplaintIngestor,
    "sqf": SQFIngestor,
    "gang_db": GangDatabaseIngestor,  # Add here
}
```

3. **Build Combined Index**:
```python
# Ingest multiple sources
records = []
for source in ["ccrb", "sqf", "gang_db"]:
    ingestor = get_ingestor(source, config)
    records.extend(ingestor.ingest(f"{source}_data.csv"))

# Build unified index
docs, meta = text_processor.prepare_documents(records)
index_builder.build_index(docs, meta)
```
